In [103]:
import numpy as np
import pandas as pd

In [104]:
def figure_out_status(x):
    # No barcode in scan file -> Status = "M"
    reference_barcode = x['Barcode Number_reference']
    scan_barcode = x['Barcode Number_scan']

    if reference_barcode is not None and scan_barcode is None:
        return 'M'

    # No barcode in reference file -> Status = "N"
    if scan_barcode is not None and reference_barcode is None:
        return 'N'

    # Change found in reference file -> Status = "C"
    reference = [col for col in x.index if col.endswith('_reference')]
    scan = [col for col in x.index if col.endswith('_scan')]
    reference_data = {col.replace('_reference', ''): x[col] for col in reference}
    scan_data = {col.replace('_scan', ''): x[col] for col in scan}

    for key in reference_data:
        if reference_data[key] != scan_data[key]:
            return 'C'

    # No changes found in reference file -> Status = "F"
    return 'F'

In [109]:
# Converting columns to a specific type helps with comparison
converters = {
    'Barcode Number': str,
    'Source': str,
    'Room': str,
    'Federal?': str,
    'Description': str,
    'Manufacturer': str,
    'Model Number': str,
    'Serial Number': str,
    'Cost Center': str,
    'COST': int,
    'Date Acquired': str,
    'Funding Code': str,
    'PO Number': str,
}

df = pd.read_excel('Scan File.xls', converters=converters)
df.insert(0, 'key', df.loc[:, 'Barcode Number'])
df

,key,Barcode Number,Source,Room,Federal?,Description,Manufacturer,Model Number,Serial Number,Cost Center,COST,Date Acquired,Funding Code,PO Number
0,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100,NaN,6001,23-5432
1,8010,8010,TipWeb,LOBBY,NaN,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,High School West,1458,2009-08-01,5101,FF-2234
2,2330,2330,NextGen,CAFE,NaN,CUP DISPENSER,HOBART,CTC2X2-5,NaN,Elementary School,385,2004-07-23,5101,510307
3,3715,3715,NextGen,602,NaN,COMPUTER TOWER,HP,360X,00045-571-601-835,Elementary School,693,2011-02-22,0,NaN
4,11762,11762,NaN,107,YES,Laptop,lenovo,100e,CD5294G9T,Middle School,NaN,NaN,NaN,NaN
5,12667,12667,NaN,208,NaN,Smartboard,Viewsonic,VS 15057,VS812847203,Elementary School,NaN,NaN,NaN,NaN


In [110]:
df2 = pd.read_excel('Reference File.xls', converters=converters)
df2.insert(0, 'key', df2.loc[:, 'Barcode Number'])
df2

,key,Barcode Number,Source,Room,Federal?,Description,Manufacturer,Model Number,Serial Number,Cost Center,COST,Date Acquired,Funding Code,PO Number
0,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100,NaN,6001,NaN
1,11965,11965,TipWeb,210,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300,NaN,6001,24-1234
2,8010,8010,TipWeb,KITCHEN,NaN,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,Middle School,1458,2009-08-01,5101,FF-2234
3,2330,2330,NextGen,CAFE,NaN,CUP DISPENSER,HOBART,CTC2X2-5,NaN,Elementary School,385,2004-07-23,5101,510307
4,3715,3715,NextGen,602,NaN,COMPUTER TOWER,DELL,OPTIPLEX 280,00045-571-601-835,Elementary School,693,2011-02-22,0,NaN


In [112]:
diffs_df = pd.merge(df, df2, suffixes=('_scan', '_reference'), how="outer", on=["key"])
diffs_df

,key,Barcode Number_scan,Source_scan,Room_scan,Federal?_scan,Description_scan,Manufacturer_scan,Model Number_scan,Serial Number_scan,Cost Center_scan,...,Federal?_reference,Description_reference,Manufacturer_reference,Model Number_reference,Serial Number_reference,Cost Center_reference,COST_reference,Date Acquired_reference,Funding Code_reference,PO Number_reference
0,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100.0,NaN,6001,NaN
1,8010,8010,TipWeb,LOBBY,NaN,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,High School West,...,NaN,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,Middle School,1458.0,2009-08-01,5101,FF-2234
2,2330,2330,NextGen,CAFE,NaN,CUP DISPENSER,HOBART,CTC2X2-5,NaN,Elementary School,...,NaN,CUP DISPENSER,HOBART,CTC2X2-5,NaN,Elementary School,385.0,2004-07-23,5101,510307
3,3715,3715,NextGen,602,NaN,COMPUTER TOWER,HP,360X,00045-571-601-835,Elementary School,...,NaN,COMPUTER TOWER,DELL,OPTIPLEX 280,00045-571-601-835,Elementary School,693.0,2011-02-22,0,NaN
4,11762,11762,NaN,107,YES,Laptop,lenovo,100e,CD5294G9T,Middle School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12667,12667,NaN,208,NaN,Smartboard,Viewsonic,VS 15057,VS812847203,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300.0,NaN,6001,24-1234


In [114]:
# Fix `NaN` values to be `None` so we can compare them
diffs_df = diffs_df.replace({np.nan: None})
diffs_df

,key,Barcode Number_scan,Source_scan,Room_scan,Federal?_scan,Description_scan,Manufacturer_scan,Model Number_scan,Serial Number_scan,Cost Center_scan,...,Federal?_reference,Description_reference,Manufacturer_reference,Model Number_reference,Serial Number_reference,Cost Center_reference,COST_reference,Date Acquired_reference,Funding Code_reference,PO Number_reference
0,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100.0,None,6001,None
1,8010,8010,TipWeb,LOBBY,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,High School West,...,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,Middle School,1458.0,2009-08-01,5101,FF-2234
2,2330,2330,NextGen,CAFE,None,CUP DISPENSER,HOBART,CTC2X2-5,None,Elementary School,...,None,CUP DISPENSER,HOBART,CTC2X2-5,None,Elementary School,385.0,2004-07-23,5101,510307
3,3715,3715,NextGen,602,None,COMPUTER TOWER,HP,360X,00045-571-601-835,Elementary School,...,None,COMPUTER TOWER,DELL,OPTIPLEX 280,00045-571-601-835,Elementary School,693.0,2011-02-22,0,None
4,11762,11762,None,107,YES,Laptop,lenovo,100e,CD5294G9T,Middle School,...,None,None,None,None,None,None,None,None,None,None
5,12667,12667,None,208,None,Smartboard,Viewsonic,VS 15057,VS812847203,Elementary School,...,None,None,None,None,None,None,None,None,None,None
6,11965,None,None,None,None,None,None,None,None,None,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300.0,None,6001,24-1234


In [115]:
# Add a new column to show the status of the row
diffs_df.insert(0, 'Status', value=diffs_df.apply(figure_out_status, axis=1))
diffs_df

,Status,key,Barcode Number_scan,Source_scan,Room_scan,Federal?_scan,Description_scan,Manufacturer_scan,Model Number_scan,Serial Number_scan,...,Federal?_reference,Description_reference,Manufacturer_reference,Model Number_reference,Serial Number_reference,Cost Center_reference,COST_reference,Date Acquired_reference,Funding Code_reference,PO Number_reference
0,C,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100.0,None,6001,None
1,C,8010,8010,TipWeb,LOBBY,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,...,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,Middle School,1458.0,2009-08-01,5101,FF-2234
2,F,2330,2330,NextGen,CAFE,None,CUP DISPENSER,HOBART,CTC2X2-5,None,...,None,CUP DISPENSER,HOBART,CTC2X2-5,None,Elementary School,385.0,2004-07-23,5101,510307
3,C,3715,3715,NextGen,602,None,COMPUTER TOWER,HP,360X,00045-571-601-835,...,None,COMPUTER TOWER,DELL,OPTIPLEX 280,00045-571-601-835,Elementary School,693.0,2011-02-22,0,None
4,N,11762,11762,None,107,YES,Laptop,lenovo,100e,CD5294G9T,...,None,None,None,None,None,None,None,None,None,None
5,N,12667,12667,None,208,None,Smartboard,Viewsonic,VS 15057,VS812847203,...,None,None,None,None,None,None,None,None,None,None
6,M,11965,None,None,None,None,None,None,None,None,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300.0,None,6001,24-1234


In [116]:
# We need to look at the status:
# For N - The row was added to the scan file, so we do nothing
# For M - The row is missing in scan file data, so we need to add it from the reference file
# For C - The row was changed in the scan file, so we just use that data, no actions needed
for column in diffs_df.columns:
    reference_column = column.replace('_scan', '_reference')
    diffs_df[column] = diffs_df.apply(lambda x: x[reference_column] if x['Status'] == 'M' else x[column], axis=1)
diffs_df

,Status,key,Barcode Number_scan,Source_scan,Room_scan,Federal?_scan,Description_scan,Manufacturer_scan,Model Number_scan,Serial Number_scan,...,Federal?_reference,Description_reference,Manufacturer_reference,Model Number_reference,Serial Number_reference,Cost Center_reference,COST_reference,Date Acquired_reference,Funding Code_reference,PO Number_reference
0,C,11944,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100.0,None,6001,None
1,C,8010,8010,TipWeb,LOBBY,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,...,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,Middle School,1458.0,2009-08-01,5101,FF-2234
2,F,2330,2330,NextGen,CAFE,None,CUP DISPENSER,HOBART,CTC2X2-5,None,...,None,CUP DISPENSER,HOBART,CTC2X2-5,None,Elementary School,385.0,2004-07-23,5101,510307
3,C,3715,3715,NextGen,602,None,COMPUTER TOWER,HP,360X,00045-571-601-835,...,None,COMPUTER TOWER,DELL,OPTIPLEX 280,00045-571-601-835,Elementary School,693.0,2011-02-22,0,None
4,N,11762,11762,None,107,YES,Laptop,lenovo,100e,CD5294G9T,...,None,None,None,None,None,None,NaN,None,None,None
5,N,12667,12667,None,208,None,Smartboard,Viewsonic,VS 15057,VS812847203,...,None,None,None,None,None,None,NaN,None,None,None
6,M,11965,11965,TipWeb,210,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,...,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300.0,None,6001,24-1234


In [117]:
# Clean up the column names to drop `_scan`
diffs_df.rename(columns={col: col.replace('_scan', '') for col in diffs_df.columns}, inplace=True)

# Remove the key column as it is no longer needed
diffs_df.drop('key', axis=1, inplace=True)

# Remove the reference columns as they are no longer needed
diffs_df.drop([col for col in diffs_df.columns if col.endswith('_reference')], axis=1, inplace=True)
diffs_df

,Status,Barcode Number,Source,Room,Federal?,Description,Manufacturer,Model Number,Serial Number,Cost Center,COST,Date Acquired,Funding Code,PO Number
0,C,11944,TipWeb,223,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX011443,High School West,1100.0,None,6001,23-5432
1,C,8010,TipWeb,LOBBY,None,CRES COR C0151FPWUA12B2083Q1,CRES COR,C0151FPWUA12B2083Q1,EAJ-J203747-044,High School West,1458.0,2009-08-01,5101,FF-2234
2,F,2330,NextGen,CAFE,None,CUP DISPENSER,HOBART,CTC2X2-5,None,Elementary School,385.0,2004-07-23,5101,510307
3,C,3715,NextGen,602,None,COMPUTER TOWER,HP,360X,00045-571-601-835,Elementary School,693.0,2011-02-22,0,None
4,N,11762,None,107,YES,Laptop,lenovo,100e,CD5294G9T,Middle School,NaN,None,None,None
5,N,12667,None,208,None,Smartboard,Viewsonic,VS 15057,VS812847203,Elementary School,NaN,None,None,None
6,M,11965,TipWeb,210,YES,ASUS CHROMEBOX CN60,ASUS,CHROMEBOX CN60,E4MSCX012824,High School East,2300.0,None,6001,24-1234
